<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Imports
import gzip
import os
import pandas as pd
import numpy as np
import torch
import basic_model
import random
from collections import Counter
from tqdm import tqdm
import ujson as json
from torch.autograd import Variable
import util
import args
import pprint

args = args.get_setup_args()
vars(args)

{'vocab_path': '/home/mouadh/Desktop/insuranceQA/V2/vocabulary',
 'label_path': '/home/mouadh/Desktop/insuranceQA/V2/InsuranceQA.label2answer.token.encoded',
 'train_path': '/home/mouadh/Desktop/insuranceQA/V2/InsuranceQA.question.anslabel.token.500.pool.solr.train.encoded',
 'test_path': '/home/mouadh/Desktop/insuranceQA/V2/InsuranceQA.question.anslabel.token.500.pool.solr.test.encoded',
 'glove_path': '/home/mouadh/Desktop/insuranceQA/glove.840B.300d/glove.840B.300d.txt',
 'glove_dim': 300,
 'glove_num_vecs': 2196017,
 'hidden_size': 100,
 'word_emb_file': '/home/mouadh/Desktop/insuranceQA/glove.840B.300d/word_embedding',
 'word2idx_file': '/home/mouadh/Desktop/insuranceQA/glove.840B.300d/word2idx',
 'seed': 0,
 'create_matrix_embedding': True,
 'lr': 0.5,
 'num_epochs': 30,
 'drop_prob': 0.2,
 'margin': 0.2,
 'batch_size': 128,
 'use_glove': False,
 'embd_size': 200}

In [2]:
# Fix the random seed to have reprodusable results
random.seed(args.seed)
np.random.seed(args.seed)
#torch.manual_seed(args.seed)
#torch.cuda.manual_seed_all(args.seed)

In [3]:
# Load answers labels and answer label with text
id2w, l2a, l2at = util.load_vocabulary(args.vocab_path, args.label_path)

# Create word to index vocabulary
w2i = {w: i for i, w in enumerate(id2w.values(), 1)}
# Add pad to the vocabulary
PAD = '<PAD>'
w2i[PAD] = 0

vocab_size = len(w2i)
args.vocab_size = vocab_size
print('vocab_size:', vocab_size)

vocab_size: 68581


In [4]:
train = util.load_data_train(args.train_path, id2w, l2at)

test = util.load_data_test(args.test_path, id2w, l2at)

In [5]:
print('n_train:', len(train))
print('n_test:', len(test))

n_train: 21325
n_test: 1988


In [6]:
if args.use_glove:
    if not args.create_matrix_embedding:
        # Create word embedding and word2idx for glove
        print("Creating word embedding")
        word_emb_mat, word2idx_dict = util.get_embedding('word', emb_file=args.glove_path,
                                        vec_size=args.glove_dim, num_vectors=args.glove_num_vecs)
        util.save(args.word_emb_file, word_emb_mat, message="word embedding")
        util.save(args.word2idx_file, word2idx_dict, message="word dictionary")
    else:
        # Get embeddings
        print('Loading word vectors embeddings...')
        word_vectors = util.torch_from_json(args.word_emb_file)
        print("Loading word to index dictionary")
        word2idx_dict = json.load(open(args.word2idx_file))
else:
    word2idx_dict = w2i

In [7]:
train[0][0]

['Is', 'Disability', 'Insurance', 'Required', 'By', 'Law', '?']

In [8]:
word2idx_dict['?']

50370

In [9]:
util.make_vector([train[0][0]], w2i, 20)

tensor([[66164, 54421, 29876, 56902, 59631, 57715, 50370,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]],
       device='cuda:0')

In [13]:
def train_model(model, data, optimizer, word2idx_dict, args, batch_size = 128):
    for epoch in range(5): #args.num_epoch:
        model.train()
        random.shuffle(data)
        losses = []
        for i, d in enumerate(tqdm(data)):
            q, pos, negs = d[0], d[1], d[2]
            #pos = process_words(pos, word2idx_dict)
            #q = process_words(q, word2idx_dict)
            
            vec_q = util.make_vector([q], word2idx_dict, len(q))
            vec_pos = util.make_vector([pos], word2idx_dict, len(pos))
            pos_sim = model(vec_q, vec_pos)

            for _ in range(100):
                neg = random.choice(negs)
                #neg = process_words(neg, word2idx_dict)
                vec_neg = util.make_vector([neg], word2idx_dict, len(neg))
                neg_sim = model(vec_q, vec_neg)
                #print('positive similarity {}, negative similarity{}'.format(pos_sim, neg_sim))
                loss = util.loss_fn(pos_sim, neg_sim, args.margin)
                if loss.data[0] != 0:
                    losses.append(loss)
                    break

            if len(losses) == batch_size or i == len(data) - 1:
                loss = torch.mean(torch.stack(losses, 0).squeeze(), 0)
                #print(loss)
                print("epoch: {}, iteration: {}, loss: {}".format(epoch, i, loss.item()))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses = []
    return model

In [14]:
def lowercase(tokens):
    return [token.lower() for token in tokens]

In [15]:
from nltk.tokenize import RegexpTokenizer
tokenizer_punct = RegexpTokenizer(r'\w+')
import itertools

def remove_special_chars(tokens):
    return list(itertools.chain(*[tokenizer_punct.tokenize(token) for token in tokens]))


In [16]:
def remove_words_not_glove(tokens, word2idx_dict):
    filteredlist = []
    for token in tokens:
        try: 
            word2idx_dict[token]
            filteredlist.append(token)
        except:
            pass
    return filteredlist

In [17]:
remove_words_not_glove(['websitelink', 'hello'], word2idx_dict)

[]

In [18]:
def process_words(tokens, word2idx_dict):
    tokens = lowercase(tokens)
    tokens = remove_special_chars(tokens)
    tokens = remove_words_not_glove(tokens, word2idx_dict)
    return tokens

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if args.use_glove:
    model = basic_model.QA_LSTM(args , word_vectors)
else: 
    model = basic_model.QA_LSTM(args)

if torch.cuda.is_available():
    model.cuda()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

In [ ]:
model = train_model(model, train, optimizer, word2idx_dict, args, args.batch_size)

  0%|          | 5/21325 [00:00<09:22, 37.90it/s]

epoch: 0


  1%|          | 123/21325 [00:02<08:39, 40.82it/s]

epoch: 0, iteration: 127, loss: 0.2012072503566742


  1%|          | 248/21325 [00:10<06:43, 52.18it/s]  

...

100%|█████████▉| 21324/21325 [50:33<00:00, 10.76it/s]

epoch: 1, iteration: 21324, loss: 0.13401077687740326


  0%|          | 0/21325 [00:00<?, ?it/s]

epoch: 2


  0%|          | 30/21325 [00:04<52:38,  6.74it/s]  

In [ ]:
def test(model, data):
    acc, total = 0, 0
    for d in data:
        q = d[0]
        print('q', ' '.join(q))
        labels = d[1]
        cands = d[2]

        # preprare answer labels
        label_indices = [cands.index(l) for l in labels if l in cands]

        # build data
        q = make_vector([q], w2i, len(q))
        cands = [label_to_ans_text[c] for c in cands] # id to text
        max_cand_len = min(args.max_sent_len, max([len(c) for c in cands]))
        cands = make_vector(cands, w2i, max_cand_len)

        # predict
        scores = [model(q, c.unsqueeze(0)).data[0] for c in cands]
        pred_idx = np.argmax(scores)
        if pred_idx in label_indices:
            print('correct')
            acc += 1
        else:
            print('wrong')
        total += 1
    print('Test Acc:', 100*acc/total, '%')

In [ ]:
test_model(model, test)